# Лабораторная работа №7 (Проведение исследований моделями семантической сегментации на наборе данных Oxford-IIIT Pet)


## Выбор начальных условий

### Выбор набора данных

Выбранный набор данных: **Oxford-IIIT Pet Dataset**

Обоснование выбора:
- **Задача семантической сегментации:** Датасет Oxford-IIIT Pet, помимо меток классов изображений, также предоставляет пиксельные аннотации (трикарты - trimaps), которые четко разделяют пиксели, относящиеся к животному (pet), к фону (background), и к области, которую можно игнорировать (border/ignore). Это делает его идеальным выбором для задачи бинарной семантической сегментации (животное vs фон).
- **Наличие аннотаций для сегментации:** Набор данных включает необходимые маски (trimaps) для обучения и оценки моделей сегментации на пиксельном уровне.
- **Умеренная сложность:** Задача сегментации на этом датасете включает вариации поз животных, освещения и текстуры фона, представляя собой реалистичную, но не чрезмерно сложную задачу для демонстрации и сравнения моделей сегментации.
- **Структурированные разделения:** Датасет предоставляет официальные разделения на тренировочный (trainval) и тестовый (test) наборы, что позволяет проводить стандартную процедуру обучения и валидации.
- **Доступность:** Датасет легко доступен для скачивания и интеграции с помощью библиотек, таких как `torchvision`, как показано в коде.
- **Бинарная задача с игнорированием:** Преобразование исходных меток маски (1, 2, 3) в целевые (1 для животного, 0 для фона, 255 для игнорируемой области) формирует четкую бинарную задачу сегментации с учетом "неизвестных" пикселей, что является распространенным сценарием в задачах сегментации.

### Выбор метрик качества

Выбранные метрики:
- **Hamming Loss (Pixel Error):**
  - Обоснование: Является обратной величиной Pixel Accuracy (1 - Pixel Accuracy). Показывает долю неправильно классифицированных пикселей. Дает простую и прямую оценку ошибки на пиксельном уровне. Важно учитывать, что эта метрика может быть завышена на классах с большим количеством пикселей (например, фон). Игнорируемые пиксели (метка 255) исключаются из расчета.
- **F1 Score (Weighted Average):**
  - Обоснование: Рассчитывается F1-оценка для каждого класса (животное, фон, исключая игнорируемый индекс 255), а затем эти оценки усредняются с весами, пропорциональными количеству истинных пикселей для каждого класса в тестовом наборе. Эта метрика учитывает как Precision (точность), так и Recall (полноту) для каждого класса и дает сбалансированную оценку производительности, смягчая влияние несбалансированности классов по площади (если один класс занимает значительно больше пикселей, чем другой).
- **mAP (mean IoU - Mean Intersection over Union) (Macro Average):**
  - Обоснование: Является стандартной метрикой для семантической сегментации. Рассчитывает IoU (отношение площади пересечения к площади объединения) для каждого класса (животное, фон, игнорируя индекс 255), а затем вычисляет среднее значение по всем классам. Это усреднение по классам (macro average) гарантирует, что каждый класс вносит равный вклад в итоговую метрику, независимо от его размера на изображениях. IoU является более строгой метрикой, чем Pixel Accuracy, так как она чувствительна к форме и точности границ сегментации.


## Подготовка и импорт библиотек

In [ ]:
%pip install pillow
%pip install pandas tabulate
%pip install torchmetrics
%pip install segmentation-models-pytorch
%pip install einops
%pip install opencv-python matplotlib scikit-image
%pip install albumentations

In [ ]:
import os
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import segmentation_models_pytorch as smp
from torchvision import datasets
import albumentations as A
from albumentations.pytorch import ToTensorV2

from einops.layers.torch import Rearrange

import matplotlib.pyplot as plt
import pandas as pd
from tabulate import tabulate

import torchmetrics

## Вспомогательные функции

In [3]:
def train_one_epoch(model, train_loader, criterion, optimizer, device):
    """
    Обучает модель в течение одной эпохи.
    """
    model.train()
    running_loss = 0.0
    for images, masks in train_loader:
        images = images.to(device)
        masks = masks.to(device)

        optimizer.zero_grad()

        outputs = model(images)

        # Убедимся, что тип данных маски - long для CrossEntropyLoss
        masks = masks.long()

        loss = criterion(outputs, masks)

        loss.backward()
        optimizer.step()

        # Суммируем потери по батчу, взвешенные по размеру батча
        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"  Train Loss: {epoch_loss:.4f}")
    return epoch_loss

In [ ]:
def evaluate(model, val_loader, num_classes, device, ignore_index=255):
    """
    Оценивает модель на валидационном наборе данных с использованием torchmetrics.
    Рассчитывает Pixel Accuracy, F1 Score (weighted), и mAP (mean IoU).
    """
    model.eval()

    # Инициализируем метрики torchmetrics, сохраняющие состояние
    # JaccardIndex с average='macro' соответствует mean IoU
    iou_metric = torchmetrics.JaccardIndex(
        task='multiclass',
        num_classes=num_classes,
        ignore_index=ignore_index,
        average='macro' # Используем 'macro' для mean IoU
    ).to(device) # Метрики также могут находиться на устройстве GPU

    # F1Score с average='weighted' учитывает дисбаланс классов на основе количества примеров
    f1_metric = torchmetrics.F1Score(
        task='multiclass',
        num_classes=num_classes,
        ignore_index=ignore_index,
        average='weighted'
    ).to(device)

    # Accuracy для расчета Pixel Accuracy
    pixel_accuracy_metric = torchmetrics.Accuracy(
        task='multiclass',
        num_classes=num_classes,
        ignore_index=ignore_index
    ).to(device)


    with torch.no_grad():
        for images, masks in val_loader:
            images = images.to(device)
            masks = masks.to(device)

            outputs = model(images)

            # Получаем предсказания (индекс класса с наивысшим скором)
            preds = torch.argmax(outputs, dim=1) # Форма: (N, H, W)

            # Убедимся, что маски имеют тип long для метрик (должны быть уже, но это хорошая практика)
            masks = masks.long()

            # Обновляем метрики текущим батчем предсказаний и ground truth
            iou_metric.update(preds, masks)
            f1_metric.update(preds, masks)
            pixel_accuracy_metric.update(preds, masks)

    # Расчет метрик
    # Вычисляем финальные значения метрик из накопленного состояния
    mAP = iou_metric.compute().item()      # mean IoU
    f1 = f1_metric.compute().item()        # Weighted F1 Score
    pixel_accuracy = pixel_accuracy_metric.compute().item() # Pixel Accuracy

    # Hamming Loss равен 1 - Pixel Accuracy
    h_loss = 1.0 - pixel_accuracy

    # Сбрасываем состояние метрик для следующего запуска оценки (важно, если вызывается evaluate несколько раз)
    iou_metric.reset()
    f1_metric.reset()
    pixel_accuracy_metric.reset()

    print(f"  Hamming Loss (Pixel Error): {h_loss:.4f}")
    print(f"  F1 Score (Weighted): {f1:.4f}")
    print(f"  mAP (mean IoU): {mAP:.4f}")

    return h_loss, f1, mAP

In [ ]:
def train_and_evaluate_model(model, train_loader, val_loader, criterion, optimizer, device, epochs, num_classes, model_name, ignore_index=255):
    """
    Обучает модель заданное количество эпох и оценивает ее после обучения.
    Возвращает финальные метрики валидации.
    """
    print(f"\n--- Training {model_name} ---")
    for epoch in range(epochs):
        print(f"EPOCH {epoch+1}/{epochs}")
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)

    print(f"Training {model_name} finished.")
    print("Evaluating on test set...")
    h_loss, f1, mAP = evaluate(model, val_loader, num_classes, device, ignore_index=ignore_index)
    print("-" * 20)

    return h_loss, f1, mAP

## Настройка параметров и датасета Oxford-IIIT Pet (Semantic Segmentation)

In [6]:
# Параметры
# Классы: 0 - фон, 1 - пиксели животного
CLASSES = ["background", "pet"]
NUM_CLASSES = len(CLASSES)
INPUT_DIM = (224, 224) # Меньший размер для ускорения обучения
BATCH_SIZE = 16
LEARNING_RATE = 0.0001 # Базовый LR для предобученных моделей
BASE_EPOCHS = 5 # Базовое количество эпох для бейзлайна и первых тестов

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# Маппинг значений маски: Original (1=pet, 2=background, 3=ignore) -> Target (1=pet, 0=background, 255=ignore)
MASK_MAPPING = { 1: 1, 2: 0, 3: 255 }
IGNORE_INDEX = 255

Using device: cuda


In [7]:
class OxfordPetSegmentationDataset(Dataset):
    def __init__(self, data_dir, image_set="trainval", transform=None, mask_mapping=None, ignore_index=255):
        self.data_dir = data_dir
        self.image_set = image_set
        self.transform = transform
        self.mask_mapping = mask_mapping if mask_mapping is not None else {}
        self.ignore_index = ignore_index

        self.image_dir = os.path.join(data_dir, 'images')
        self.mask_dir = os.path.join(data_dir, 'annotations', 'trimaps')

        # Загружаем имена файлов изображений из официальных сплитов
        list_file = os.path.join(data_dir, 'annotations', f'{image_set}.txt')

        if not os.path.exists(list_file):
             raise FileNotFoundError(f"Файл сплита {list_file} не найден. Пожалуйста, скачайте датасет.")

        image_basenames = []
        with open(list_file, 'r') as f:
            for line in f:
                 line = line.strip()
                 if line:
                     image_basenames.append(line.split()[0])

        self.items = []
        for basename in image_basenames:
            img_path = os.path.join(self.image_dir, basename + '.jpg')
            mask_path = os.path.join(self.mask_dir, basename + '.png')

            if os.path.exists(img_path) and os.path.exists(mask_path):
                 self.items.append((img_path, mask_path))
            else:
                 print(f"Отсутствует файл(ы) для {basename}: {img_path} или {mask_path}")

        print(f"Загружено {len(self.items)} объектов для сплита {image_set}.")

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        img_path, mask_path = self.items[idx]

        # Загружаем изображение (Убедимся в формате RGB)
        img = np.array(Image.open(img_path).convert('RGB'))

        # Загружаем маску (оттенки серого). Тримапы имеют значения 1, 2, 3.
        mask = np.array(Image.open(mask_path).convert('L'), dtype=np.uint8)

        # Применяем маппинг маски: Original (1=pet, 2=background, 3=ignore) -> Target (1=pet, 0=background, 255=ignore)
        processed_mask = np.full_like(mask, self.ignore_index, dtype=np.uint8)

        # Применяем маппинг для известных значений
        for original_value, target_value in self.mask_mapping.items():
             processed_mask[mask == original_value] = target_value

        if 3 in self.mask_mapping:
             processed_mask[mask == 3] = self.mask_mapping[3]


        if self.transform:
            augmented = self.transform(image=img, mask=processed_mask)
            img = augmented['image'] # Должен быть тензором (C, H, W) из-за ToTensorV2
            mask = augmented['mask'] # Должен быть тензором (H, W) из-за ToTensorV2 (transpose_mask=False по умолчанию)

        # Albumentations ToTensorV2 по умолчанию возвращает маску как float32. Преобразуем в long.
        mask = mask.long()

        return img, mask

In [8]:
# Определение пайплайнов аугментации и преобразования

# Базовые преобразования (для валидации и не-аугментированного обучения)
base_transform = A.Compose([
    A.Resize(*INPUT_DIM),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(), # Преобразует изображение в тензор C,H,W float; маску в тензор H,W float
])

# Аугментации для тренировочного набора
augmentations = A.Compose([
    A.Resize(*INPUT_DIM), # Применяем Resize первым, чтобы другие аугментации работали с правильными размерами
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.1),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=5, p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(), # Преобразует изображение в тензор C,H,W float; маску в тензор H,W float
])

/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:111: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [9]:
# Загрузка датасетов и создание DataLoader'ов (для базового обучения)
datasets.OxfordIIITPet(root='./data', split='trainval', download=True)

print("Loading datasets (base transforms)...")
train_dataset_base = OxfordPetSegmentationDataset(
    data_dir="./data/oxford-iiit-pet",
    image_set="trainval", # Oxford имеет сплиты trainval и test
    transform=base_transform,
    mask_mapping=MASK_MAPPING,
    ignore_index=IGNORE_INDEX
)
test_dataset = OxfordPetSegmentationDataset(
    data_dir="./data/oxford-iiit-pet",
    image_set="test",
    transform=base_transform,
    mask_mapping=MASK_MAPPING,
    ignore_index=IGNORE_INDEX
)

train_loader_base = DataLoader(
    dataset=train_dataset_base,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2 # Рассмотрите увеличение num_workers, если CPU является узким местом
)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2
)

print(f"Train loader size: {len(train_loader_base.dataset)}")
print(f"Test loader size: {len(test_loader.dataset)}")

Loading datasets (base transforms)...
Loaded 3680 items for trainval set.
Loaded 3669 items for test set.
Train loader size: 3680
Test loader size: 3669


## Бейзлайн эксперименты (готовые модели из библиотеки)

Инициализация моделей Unet и Segformer из `segmentation-models-pytorch`

In [10]:
# Инициализация Unet
model_unet_base = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet", # Используем предобученные веса
    in_channels=3,
    classes=NUM_CLASSES,
    activation=None # None для CrossEntropyLoss (функция активации применяется внутри лосса)
)
model_unet_base.to(DEVICE);

criterion = nn.CrossEntropyLoss(ignore_index=IGNORE_INDEX)
optimizer_unet_base = optim.Adam(model_unet_base.parameters(), lr=LEARNING_RATE)

In [11]:
# Инициализация Segformer
# Segformer обычно использует энкодеры MixTransformer (mit_b0 до mit_b5).
# Resnet34 - стандартный CNN энкодер. Давайте используем типичный энкодер Segformer.
# Используем mit_b0, так как он относительно небольшой.
model_segformer_base = smp.Segformer(
    encoder_name="mit_b0",
    encoder_weights="imagenet",
    in_channels=3,
    classes=NUM_CLASSES,
    # Segformer в smp по умолчанию использует Softmax в конце декодера,
    # поэтому activation=None не требуется, если используется CrossEntropyLoss.
    # Если бы использовались другие лоссы, мог бы потребоваться 'sigmoid'.
)
model_segformer_base.to(DEVICE);

optimizer_segformer_base = optim.Adam(model_segformer_base.parameters(), lr=LEARNING_RATE)

Обучение и оценка базовой модели Unet

In [ ]:
print("\n--- Evaluating Untrained Unet Baseline")
h_loss_unet_untrained, f1_unet_untrained, mAP_unet_untrained = evaluate(
    model=model_unet_base, # Используем модель сразу после инициализации
    val_loader=test_loader,
    num_classes=NUM_CLASSES,
    device=DEVICE,
    ignore_index=IGNORE_INDEX
)

print("\n--- Evaluating Untrained Segformer Baseline ---")
h_loss_segformer_untrained, f1_segformer_untrained, mAP_segformer_untrained = evaluate(
    model=model_segformer_base, # Используем модель сразу после инициализации
    val_loader=test_loader,
    num_classes=NUM_CLASSES,
    device=DEVICE,
    ignore_index=IGNORE_INDEX
)


--- Evaluating Untrained Unet Baseline ---
  Hamming Loss (Pixel Error): 0.4115
  F1 Score (Weighted): 0.5718
  mAP (mean IoU): 0.3656

--- Evaluating Untrained Segformer Baseline ---
  Hamming Loss (Pixel Error): 0.5706
  F1 Score (Weighted): 0.4311
  mAP (mean IoU): 0.2733


In [14]:
h_loss_unet_base, f1_unet_base, mAP_unet_base = train_and_evaluate_model(
    model=model_unet_base,
    train_loader=train_loader_base,
    val_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer_unet_base,
    device=DEVICE,
    epochs=BASE_EPOCHS,
    num_classes=NUM_CLASSES,
    ignore_index=IGNORE_INDEX,
    model_name="Unet (base)"
)


--- Training Unet (base) ---
EPOCH 1/5
  Train Loss: 0.1729
EPOCH 2/5
  Train Loss: 0.0564
EPOCH 3/5
  Train Loss: 0.0371
EPOCH 4/5
  Train Loss: 0.0277
EPOCH 5/5
  Train Loss: 0.0243
Training Unet (base) finished.
Evaluating on test set...
  Hamming Loss (Pixel Error): 0.0145
  F1 Score (Weighted): 0.9855
  mAP (mean IoU): 0.9684
--------------------


Обучение и оценка базовой модели Segformer

In [15]:
h_loss_segformer_base, f1_segformer_base, mAP_segformer_base = train_and_evaluate_model(
    model=model_segformer_base,
    train_loader=train_loader_base,
    val_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer_segformer_base,
    device=DEVICE,
    epochs=BASE_EPOCHS,
    num_classes=NUM_CLASSES,
    ignore_index=IGNORE_INDEX,
    model_name="Segformer (base)"
)


--- Training Segformer (base) ---
EPOCH 1/5
  Train Loss: 0.1076
EPOCH 2/5
  Train Loss: 0.0512
EPOCH 3/5
  Train Loss: 0.0403
EPOCH 4/5
  Train Loss: 0.0281
EPOCH 5/5
  Train Loss: 0.0241
Training Segformer (base) finished.
Evaluating on test set...
  Hamming Loss (Pixel Error): 0.0133
  F1 Score (Weighted): 0.9868
  mAP (mean IoU): 0.9711
--------------------


## Улучшение бейзлайна

### Формулирование гипотез

На основе предыдущей лабораторной работы по классификации, где аугментация данных и подбор гиперпараметров значительно улучшили качество моделей, сформулируем аналогичные гипотезы для семантической сегментации.

#### Гипотеза 1: Аугментация данных

Цель: Повышение генерализирующей способности моделей семантической сегментации. Поскольку сегментация требует сохранения пространственной согласованности между изображением и маской, аугментации должны применяться синхронно к обоим.

Предполагаемый эффект: Уменьшение переобучения на тренировочном наборе, повышение устойчивости к вариациям в тестовых данных (поворот, масштабирование, изменение освещенности).

#### Гипотеза 2: Подбор гиперпараметров и оптимизатора

Цель: Найти более эффективные параметры обучения для дообучения предобученных моделей.

Идеи для реализации:
*   Увеличение количества эпох обучения: Для более тщательной настройки весов на целевом датасете.
*   Снижение Learning Rate: Меньший шаг оптимизации для тонкой настройки предобученных весов.
*   Смена оптимизатора: Возможно, AdamW, часто более эффективный для моделей на основе трансформеров (как Segformer), также подойдет и для сверточных моделей.


### Проверка гипотез

#### Гипотеза 1: Аугментация данных

In [17]:
# Создание датасета с аугментацией для тренировочного набора
print("Loading training dataset with augmentations...")
train_dataset_aug = OxfordPetSegmentationDataset(
    data_dir="./data/oxford-iiit-pet",
    image_set="trainval",
    transform=augmentations,
    mask_mapping=MASK_MAPPING,
    ignore_index=IGNORE_INDEX
)

train_loader_aug = DataLoader(
    dataset=train_dataset_aug,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2
)
print(f"Augmented train loader size: {len(train_loader_aug.dataset)}")

Loading training dataset with augmentations...
Loaded 3680 items for trainval set.
Augmented train loader size: 3680


Обучение и оценка Unet с аугментацией (используем базовые HPs)

In [18]:
# Инициализация Unet (начинаем с чистого листа)
model_unet_aug = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=NUM_CLASSES,
    activation=None
)
model_unet_aug.to(DEVICE);
# Используем базовый LR и Adam
optimizer_unet_aug = optim.Adam(model_unet_aug.parameters(), lr=LEARNING_RATE)

h_loss_unet_aug, f1_unet_aug, mAP_unet_aug = train_and_evaluate_model(
    model=model_unet_aug,
    train_loader=train_loader_aug,
    val_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer_unet_aug,
    device=DEVICE,
    epochs=BASE_EPOCHS, # Столько же эпох, как в бейзлайне
    num_classes=NUM_CLASSES,
    ignore_index=IGNORE_INDEX,
    model_name="Unet (aug)"
)


--- Training Unet (aug) ---
EPOCH 1/5
  Train Loss: 0.1795
EPOCH 2/5
  Train Loss: 0.0633
EPOCH 3/5
  Train Loss: 0.0488
EPOCH 4/5
  Train Loss: 0.0410
EPOCH 5/5
  Train Loss: 0.0305
Training Unet (aug) finished.
Evaluating on test set...
  Hamming Loss (Pixel Error): 0.0157
  F1 Score (Weighted): 0.9843
  mAP (mean IoU): 0.9658
--------------------


Обучение и оценка Segformer с аугментацией (используем базовые HPs)

In [19]:
# Инициализация Segformer (начинаем с чистого листа)
model_segformer_aug = smp.Segformer(
    encoder_name="mit_b0",
    encoder_weights="imagenet",
    in_channels=3,
    classes=NUM_CLASSES,
)
model_segformer_aug.to(DEVICE);
# Используем базовый LR и Adam
optimizer_segformer_aug = optim.Adam(model_segformer_aug.parameters(), lr=LEARNING_RATE)

h_loss_segformer_aug, f1_segformer_aug, mAP_segformer_aug = train_and_evaluate_model(
    model=model_segformer_aug,
    train_loader=train_loader_aug,
    val_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer_segformer_aug,
    device=DEVICE,
    epochs=BASE_EPOCHS, # Столько же эпох, как в бейзлайне
    num_classes=NUM_CLASSES,
    ignore_index=IGNORE_INDEX,
    model_name="Segformer (aug)"
)


--- Training Segformer (aug) ---
EPOCH 1/5
  Train Loss: 0.1019
EPOCH 2/5
  Train Loss: 0.0582
EPOCH 3/5
  Train Loss: 0.0465
EPOCH 4/5
  Train Loss: 0.0402
EPOCH 5/5
  Train Loss: 0.0331
Training Segformer (aug) finished.
Evaluating on test set...
  Hamming Loss (Pixel Error): 0.0133
  F1 Score (Weighted): 0.9867
  mAP (mean IoU): 0.9710
--------------------


#### Гипотеза 2: Подбор гиперпараметров и оптимизатора

In [20]:
# Увеличиваем количество эпох и уменьшаем LR для проверки гипотезы
HP_TUNING_EPOCHS = 10 # Увеличиваем количество эпох
HP_TUNING_LEARNING_RATE = 0.00005 # Уменьшаем LR

# Используем базовый датасет без аугментации для изоляции эффекта HPs
train_loader_hp = train_loader_base

Обучение и оценка Unet с новыми HPs (без аугментации)

In [21]:
# Инициализация Unet (начинаем с чистого листа)
model_unet_hp = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=NUM_CLASSES,
    activation=None
)
model_unet_hp.to(DEVICE);
optimizer_unet_hp = optim.Adam(model_unet_hp.parameters(), lr=HP_TUNING_LEARNING_RATE)

h_loss_unet_hp, f1_unet_hp, mAP_unet_hp = train_and_evaluate_model(
    model=model_unet_hp,
    train_loader=train_loader_hp,
    val_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer_unet_hp,
    device=DEVICE,
    epochs=HP_TUNING_EPOCHS,
    num_classes=NUM_CLASSES,
    ignore_index=IGNORE_INDEX,
    model_name="Unet (HP)"
)


--- Training Unet (HP) ---
EPOCH 1/10
  Train Loss: 0.2108
EPOCH 2/10
  Train Loss: 0.0661
EPOCH 3/10
  Train Loss: 0.0387
EPOCH 4/10
  Train Loss: 0.0252
EPOCH 5/10
  Train Loss: 0.0189
EPOCH 6/10
  Train Loss: 0.0152
EPOCH 7/10
  Train Loss: 0.0114
EPOCH 8/10
  Train Loss: 0.0101
EPOCH 9/10
  Train Loss: 0.0178
EPOCH 10/10
  Train Loss: 0.0084
Training Unet (HP) finished.
Evaluating on test set...
  Hamming Loss (Pixel Error): 0.0127
  F1 Score (Weighted): 0.9873
  mAP (mean IoU): 0.9721
--------------------


Обучение и оценка Segformer с новыми HPs (без аугментации)

In [22]:
# Инициализация Segformer (начинаем с чистого листа)
model_segformer_hp = smp.Segformer(
    encoder_name="mit_b0",
    encoder_weights="imagenet",
    in_channels=3,
    classes=NUM_CLASSES,
)
model_segformer_hp.to(DEVICE);
# Используем AdamW для трансформера с пониженным LR
optimizer_segformer_hp = optim.AdamW(model_segformer_hp.parameters(), lr=HP_TUNING_LEARNING_RATE, weight_decay=0.01)

h_loss_segformer_hp, f1_segformer_hp, mAP_segformer_hp = train_and_evaluate_model(
    model=model_segformer_hp,
    train_loader=train_loader_hp,
    val_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer_segformer_hp,
    device=DEVICE,
    epochs=HP_TUNING_EPOCHS,
    num_classes=NUM_CLASSES,
    ignore_index=IGNORE_INDEX,
    model_name="Segformer (HP)"
)


--- Training Segformer (HP) ---
EPOCH 1/10
  Train Loss: 0.1405
EPOCH 2/10
  Train Loss: 0.0599
EPOCH 3/10
  Train Loss: 0.0466
EPOCH 4/10
  Train Loss: 0.0366
EPOCH 5/10
  Train Loss: 0.0295
EPOCH 6/10
  Train Loss: 0.0269
EPOCH 7/10
  Train Loss: 0.0245
EPOCH 8/10
  Train Loss: 0.0213
EPOCH 9/10
  Train Loss: 0.0208
EPOCH 10/10
  Train Loss: 0.0180
Training Segformer (HP) finished.
Evaluating on test set...
  Hamming Loss (Pixel Error): 0.0125
  F1 Score (Weighted): 0.9875
  mAP (mean IoU): 0.9727
--------------------


### Формирование улучшенного бейзлайна

На основе результатов проверки гипотез, мы объединим наиболее успешные техники для формирования "улучшенного" бейзлайна.

Ожидается, что комбинация **аугментации данных** с **улучшенными гиперпараметрами (пониженный LR, увеличенные эпохи, AdamW для Segformer)** даст наилучший результат для предобученных моделей из `segmentation-models-pytorch`.

In [23]:
# Определяем окончательные параметры для улучшенного бейзлайна
FINAL_EPOCHS = 10 # Увеличиваем количество эпох для финального прогона
FINAL_LEARNING_RATE = HP_TUNING_LEARNING_RATE # Используем пониженный LR

# Используем датасет с аугментацией
train_loader_improved = train_loader_aug

print("Настройки улучшенного бейзлайна: Аугментация, LR={FINAL_LEARNING_RATE}, Epochs={FINAL_EPOCHS}".format(FINAL_LEARNING_RATE=FINAL_LEARNING_RATE, FINAL_EPOCHS=FINAL_EPOCHS))

Настройки улучшенного бейзлайна: Аугментация, LR=5e-05, Epochs=10


Обучение и оценка улучшенной модели Unet

In [24]:
# Инициализация Unet (начинаем с чистого листа)
model_unet_improved = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=NUM_CLASSES,
    activation=None
)
model_unet_improved.to(DEVICE);
optimizer_unet_improved = optim.Adam(model_unet_improved.parameters(), lr=FINAL_LEARNING_RATE)

h_loss_unet_improved, f1_unet_improved, mAP_unet_improved = train_and_evaluate_model(
    model=model_unet_improved,
    train_loader=train_loader_improved,
    val_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer_unet_improved,
    device=DEVICE,
    epochs=FINAL_EPOCHS,
    num_classes=NUM_CLASSES,
    ignore_index=IGNORE_INDEX,
    model_name="Unet (improved)"
)


--- Training Unet (improved) ---
EPOCH 1/10
  Train Loss: 0.2295
EPOCH 2/10
  Train Loss: 0.0830
EPOCH 3/10
  Train Loss: 0.0561
EPOCH 4/10
  Train Loss: 0.0409
EPOCH 5/10
  Train Loss: 0.0351
EPOCH 6/10
  Train Loss: 0.0298
EPOCH 7/10
  Train Loss: 0.0247
EPOCH 8/10
  Train Loss: 0.0200
EPOCH 9/10
  Train Loss: 0.0184
EPOCH 10/10
  Train Loss: 0.0147
Training Unet (improved) finished.
Evaluating on test set...
  Hamming Loss (Pixel Error): 0.0113
  F1 Score (Weighted): 0.9887
  mAP (mean IoU): 0.9752
--------------------


Обучение и оценка улучшенной модели Segformer

In [25]:
# Инициализация Segformer (начинаем с чистого листа)
model_segformer_improved = smp.Segformer(
    encoder_name="mit_b0",
    encoder_weights="imagenet",
    in_channels=3,
    classes=NUM_CLASSES,
)
model_segformer_improved.to(DEVICE);
# Используем AdamW
optimizer_segformer_improved = optim.AdamW(model_segformer_improved.parameters(), lr=FINAL_LEARNING_RATE, weight_decay=0.01)

h_loss_segformer_improved, f1_segformer_improved, mAP_segformer_improved = train_and_evaluate_model(
    model=model_segformer_improved,
    train_loader=train_loader_improved,
    val_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer_segformer_improved,
    device=DEVICE,
    epochs=FINAL_EPOCHS,
    num_classes=NUM_CLASSES,
    ignore_index=IGNORE_INDEX,
    model_name="Segformer (improved)"
)


--- Training Segformer (improved) ---
EPOCH 1/10
  Train Loss: 0.1253
EPOCH 2/10
  Train Loss: 0.0619
EPOCH 3/10
  Train Loss: 0.0527
EPOCH 4/10
  Train Loss: 0.0460
EPOCH 5/10
  Train Loss: 0.0384
EPOCH 6/10
  Train Loss: 0.0366
EPOCH 7/10
  Train Loss: 0.0331
EPOCH 8/10
  Train Loss: 0.0296
EPOCH 9/10
  Train Loss: 0.0289
EPOCH 10/10
  Train Loss: 0.0275
Training Segformer (improved) finished.
Evaluating on test set...
  Hamming Loss (Pixel Error): 0.0119
  F1 Score (Weighted): 0.9881
  mAP (mean IoU): 0.9738
--------------------


### Сравнение результатов бейзлайн моделей

In [26]:
results = {
    'Model': ['Unet (base)', 'Unet (aug)', 'Unet (HP)', 'Unet (improved)',
              'Segformer (base)', 'Segformer (aug)', 'Segformer (HP)', 'Segformer (improved)'],
    'Hamming Loss (Pixel Error)': [h_loss_unet_base, h_loss_unet_aug, h_loss_unet_hp, h_loss_unet_improved,
                     h_loss_segformer_base, h_loss_segformer_aug, h_loss_segformer_hp, h_loss_segformer_improved],
    'F1 Score (Weighted)': [f1_unet_base, f1_unet_aug, f1_unet_hp, f1_unet_improved,
                 f1_segformer_base, f1_segformer_aug, f1_segformer_hp, f1_segformer_improved],
    'mAP (mean IoU)': [mAP_unet_base, mAP_unet_aug, mAP_unet_hp, mAP_unet_improved,
                       mAP_segformer_base, mAP_segformer_aug, mAP_segformer_hp, mAP_segformer_improved]
}

df_baseline_comparison = pd.DataFrame(results).round(4)
print("\nСравнение результатов предобученных моделей:")
print(tabulate(df_baseline_comparison, headers='keys', tablefmt='pipe'))


Сравнение результатов предобученных моделей:
|    | Model                |   Hamming Loss (Pixel Error) |   F1 Score (Weighted) |   mAP (mean IoU) |
|---:|:---------------------|-----------------------------:|----------------------:|-----------------:|
|  0 | Unet (base)          |                       0.0145 |                0.9855 |           0.9684 |
|  1 | Unet (aug)           |                       0.0157 |                0.9843 |           0.9658 |
|  2 | Unet (HP)            |                       0.0127 |                0.9873 |           0.9721 |
|  3 | Unet (improved)      |                       0.0113 |                0.9887 |           0.9752 |
|  4 | Segformer (base)     |                       0.0133 |                0.9868 |           0.9711 |
|  5 | Segformer (aug)      |                       0.0133 |                0.9867 |           0.971  |
|  6 | Segformer (HP)       |                       0.0125 |                0.9875 |           0.9727 |
|  7 | Segformer (

### Выводы по бейзлайн моделям

На основе сравнения метрик в таблице можно сделать следующие выводы:

*   Базовые модели (Unet base, Segformer base), обученные всего 5 эпох с LR 0.0001, показывают приемлемые, но не выдающиеся результаты. Unet base достиг mAP 0.9684, Segformer base - mAP 0.9711.
*   Аугментация данных (Unet aug, Segformer aug) на этом этапе (с базовыми HPs и 5 эпохами) не привела к существенному улучшению, а в случае Unet даже немного ухудшила метрики (mAP 0.9658 для Unet aug, 0.9710 для Segformer aug). Возможно, для полного проявления эффекта аугментации требуется больше эпох обучения или другие гиперпараметры.
*   Подбор гиперпараметров (Увеличение эпох до 10 и снижение LR до 0.00005, а также смена оптимизатора на AdamW для Segformer) оказал более существенное влияние (Unet HP, Segformer HP). Метрики значительно выросли по сравнению с базовым бейзлайном: Unet HP достиг mAP 0.9721, а Segformer HP - mAP 0.9727. Это подтверждает Гипотезу 2 и указывает на то, что предобученным моделям нужно больше времени и более тонкая настройка для адаптации к целевому датасету.
*   Лучшие результаты для обеих архитектур достигнуты при комбинации аугментации и улучшенных гиперпараметров (Unet improved, Segformer improved). Unet improved достиг mAP 0.9752, а Segformer improved - mAP 0.9738. Это ожидаемо, поскольку эти техники дополняют друг друга: аугментация помогает с генерализацией, а тонкая настройка обучения позволяет модели лучше использовать увеличенный набор данных.
*   Среди предобученных моделей, Unet (resnet34) в своем лучшем варианте (Unet improved с mAP 0.9752) показал немного лучшие результаты по сравнению с Segformer (mit_b0) в его лучшем варианте (Segformer improved с mAP 0.9738) на данном датасете с использованными параметрами. Это может говорить о том, что для этой задачи и с данным энкодером Unet оказался чуть эффективнее, хотя Segformer также показывает очень высокие метрики.


## Имплементация упрощенных алгоритмов машинного обучения

### Имплементации моделей

Упрощенная имплементация Unet (только 2 блока энкодера и 2 блока декодера, без skip-connections для максимального упрощения)

In [ ]:
class SimpleUNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SimpleUNet, self).__init__()

        self.conv1 = self.conv_block(in_channels, 32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = self.conv_block(32, 64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) # Выход H/4, W/4

        # Бутылочное горлышко (простое)
        self.bottleneck = self.conv_block(64, 128)

        # Декодер - апсемплинг с помощью ConvTranspose
        self.upconv2 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec_conv2 = self.conv_block(64, 64)

        self.upconv1 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)
        self.dec_conv1 = self.conv_block(32, 32)

        # Выходной слой
        self.out_conv = nn.Conv2d(32, out_channels, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # Энкодер
        x1 = self.conv1(x)
        x = self.pool1(x1)

        x2 = self.conv2(x)
        x = self.pool2(x2) # Выход H/4 x W/4
        x = self.bottleneck(x)

        # Декодер (без skip-соединений)
        x = self.upconv2(x) # Апсемплинг до H/2 x W/2
        # Если используются skip-соединения, здесь нужно объединить x с x2
        x = self.dec_conv2(x)

        x = self.upconv1(x) # Апсемплинг до H x W
        # Если используются skip-соединения, здесь нужно объединить x с x1
        x = self.dec_conv1(x)

        # Выход
        out = self.out_conv(x)

        return out

Упрощенная имплементация Vision Transformer для сегментации

In [ ]:
# Базовый блок Transformer Encoder (упрощенный от стандартного ViT)
class TransformerEncoderBlock(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        # Используем стандартный nn.MultiheadAttention
        self.attn = nn.MultiheadAttention(dim, num_heads, dropout=dropout, batch_first=True)
        self.norm2 = nn.LayerNorm(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = nn.Sequential(
            nn.Linear(dim, mlp_hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_hidden_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        # Остаточные соединения
        identity = x
        x = self.norm1(x)
        # MultiheadAttention возвращает (выход, веса) - берем только выход
        x, _ = self.attn(x, x, x)
        x = identity + x # Добавляем остаточное соединение

        identity = x
        x = self.norm2(x)
        x = self.mlp(x)
        x = identity + x # Добавляем остаточное соединение
        return x

class SimpleViTSegmentation(nn.Module):
    def __init__(self, image_size, patch_size, num_classes, dim, depth, num_heads, mlp_ratio, channels=3):
        super().__init__()

        assert image_size % patch_size == 0, 'Размеры изображения должны делиться на размер патча.'

        self.image_size = image_size
        self.patch_size = patch_size
        self.num_classes = num_classes # Сохраняем num_classes
        grid_size = image_size // patch_size
        num_patches = grid_size * grid_size
        patch_dim = channels * patch_size ** 2

        self.patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=patch_size, p2=patch_size), # (B, num_patches, patch_dim)
            nn.LayerNorm(patch_dim), # Добавляем LayerNorm перед линейной проекцией
            nn.Linear(patch_dim, dim), # Проецируем признаки патча в размерность модели
            # nn.LayerNorm(dim), # Опциональный LayerNorm после проекции
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches, dim))

        # Энкодер Трансформера
        self.transformer_encoder = nn.Sequential(*[
            TransformerEncoderBlock(dim, num_heads, mlp_ratio)
            for _ in range(depth)
        ])

        # Голова сегментации (Декодер)
        self.segmentation_head = nn.Sequential(
            nn.LayerNorm(dim), # Нормализация перед головой
            nn.Linear(dim, self.num_classes) # Проецируем размерность токена в логиты классов для каждого патча
        )

    def forward(self, img):
        b, c, h, w = img.shape

        if h != self.image_size or w != self.image_size:
             # Это должно обрабатываться преобразованиями датасета, но проверка полезна.
             raise ValueError(f"Размер входного изображения {(h, w)} не соответствует ожидаемому {self.image_size}")

        # 1. Встраивание патчей
        x = self.patch_embedding(img) # (B, num_patches, dim)

        # 2. Добавляем позиционное встраивание
        x = x + self.pos_embedding # (B, num_patches, dim)

        # 3. Энкодер Трансформера
        x = self.transformer_encoder(x) # (B, num_patches, dim)

        # 4. Голова сегментации
        x = self.segmentation_head(x) # (B, num_patches, num_classes)

        # 5. Меняем форму и апсемплим до исходного размера изображения
        grid_size = self.image_size // self.patch_size

        # Проверяем, соответствует ли количество патчей квадрату размера сетки
        assert x.shape[1] == grid_size * grid_size, "Несоответствие количества патчей"

        x = x.view(b, grid_size, grid_size, self.num_classes) # (B, grid_h, grid_w, num_classes)
        x = x.permute(0, 3, 1, 2) # (B, num_classes, grid_h, grid_w)

        # Апсемплинг до исходных размеров изображения (H, W)
        x = nn.functional.interpolate(x, size=(h, w), mode='bilinear', align_corners=False)

        return x # (B, num_classes, H, W)

### Обучение упрощенных моделей (Базовый вариант)

Для базового сравнения имплементированных моделей с предобученными бейзлайнами, обучим их с нуля, используя **стандартный датасет без аугментации** и **базовые гиперпараметры**, аналогичные тем, что использовались для первых прогонов предобученных моделей (хотя для обучения с нуля часто требуются другие параметры). Это позволит увидеть "сырую" производительность простых архитектур без преимуществ предобучения или оптимизированных техник обучения.

*   Количество эпох: 5
*   Learning Rate: 0.001 (немного выше, чем базовый для предобученных, часто нужно больше для обучения с нуля)

In [29]:
# Используем стандартный (не аугментированный) даталоадер
train_loader_simple_base = train_loader_base

SIMPLE_MODEL_BASE_LR = 0.001 # Начальный LR для обучения с нуля

# Инициализация Simple UNet (начинаем с чистого листа)
model_unet_simple_base = SimpleUNet(in_channels=3, out_channels=NUM_CLASSES).to(DEVICE)
optimizer_unet_simple_base = optim.Adam(model_unet_simple_base.parameters(), lr=SIMPLE_MODEL_BASE_LR)

h_loss_unet_simple_base, f1_unet_simple_base, mAP_unet_simple_base = train_and_evaluate_model(
    model=model_unet_simple_base,
    train_loader=train_loader_simple_base,
    val_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer_unet_simple_base,
    device=DEVICE,
    epochs=BASE_EPOCHS, # Столько же эпох, как базовый бейзлайн
    num_classes=NUM_CLASSES,
    ignore_index=IGNORE_INDEX,
    model_name="Simple UNet (Impl base)"
)


--- Training Simple UNet (Impl base) ---
EPOCH 1/5
  Train Loss: 0.5843
EPOCH 2/5
  Train Loss: 0.5103
EPOCH 3/5
  Train Loss: 0.4862
EPOCH 4/5
  Train Loss: 0.4372
EPOCH 5/5
  Train Loss: 0.4164
Training Simple UNet (Impl base) finished.
Evaluating on test set...
  Hamming Loss (Pixel Error): 0.1588
  F1 Score (Weighted): 0.8405
  mAP (mean IoU): 0.7020
--------------------


Обучение и оценка Simple ViT (базовый)

In [44]:
# Параметры Simple ViT (выбираем достаточно небольшие)
vit_patch_size = 16
vit_dim = 128 # Размерность токенов
vit_depth = 3 # Количество слоев трансформера
vit_heads = 4 # Количество голов внимания
vit_mlp_ratio = 4.0 # Размерность скрытого слоя в MLP относительно dim

model_vit_simple_base = SimpleViTSegmentation(
    image_size=INPUT_DIM[0],
    patch_size=vit_patch_size,
    num_classes=NUM_CLASSES,
    dim=vit_dim,
    depth=vit_depth,
    num_heads=vit_heads,
    mlp_ratio=vit_mlp_ratio,
    channels=3
).to(DEVICE)

# Используем базовый LR и Adam
optimizer_vit_simple_base = optim.Adam(model_vit_simple_base.parameters(), lr=SIMPLE_MODEL_BASE_LR)

h_loss_vit_simple_base, f1_vit_simple_base, mAP_vit_simple_base = train_and_evaluate_model(
    model=model_vit_simple_base,
    train_loader=train_loader_simple_base,
    val_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer_vit_simple_base,
    device=DEVICE,
    epochs=BASE_EPOCHS, # Столько же эпох, как базовый бейзлайн
    num_classes=NUM_CLASSES,
    ignore_index=IGNORE_INDEX,
    model_name="Simple ViT (Impl base)"
)


--- Training Simple ViT (Impl base) ---
EPOCH 1/5
  Train Loss: 0.4044
EPOCH 2/5
  Train Loss: 0.3413
EPOCH 3/5
  Train Loss: 0.3320
EPOCH 4/5
  Train Loss: 0.3093
EPOCH 5/5
  Train Loss: 0.2906
Training Simple ViT (Impl base) finished.
Evaluating on test set...
  Hamming Loss (Pixel Error): 0.1249
  F1 Score (Weighted): 0.8729
  mAP (mean IoU): 0.7528
--------------------


### Сравнение результатов упрощенных моделей с базовыми (на стандартных параметрах)

In [45]:
results_simple_base = {
    'Model': ['Unet (base)', 'Segformer (base)', 'Simple UNet (Impl base)', 'Simple ViT (Impl base)'],
    'Hamming Loss (Pixel Error)': [h_loss_unet_base, h_loss_segformer_base, h_loss_unet_simple_base, h_loss_vit_simple_base],
    'F1 Score (Weighted)': [f1_unet_base, f1_segformer_base, f1_unet_simple_base, f1_vit_simple_base],
    'mAP (mean IoU)': [mAP_unet_base, mAP_segformer_base, mAP_unet_simple_base, mAP_vit_simple_base]
}

df_simple_base_comparison = pd.DataFrame(results_simple_base).round(4)
print("\nСравнение имплементированных моделей (базовое обучение) с предобученными бейзлайнами:")
print(tabulate(df_simple_base_comparison, headers='keys', tablefmt='pipe'))


Сравнение имплементированных моделей (базовое обучение) с предобученными бейзлайнами:
|    | Model                   |   Hamming Loss (Pixel Error) |   F1 Score (Weighted) |   mAP (mean IoU) |
|---:|:------------------------|-----------------------------:|----------------------:|-----------------:|
|  0 | Unet (base)             |                       0.0145 |                0.9855 |           0.9684 |
|  1 | Segformer (base)        |                       0.0133 |                0.9868 |           0.9711 |
|  2 | Simple UNet (Impl base) |                       0.1588 |                0.8405 |           0.7020 |
|  3 | Simple ViT (Impl base)  |                       0.1249 |                0.8729 |           0.7528 |


### Выводы по упрощенным моделям (базовый вариант)

На основе сравнения метрик можно сделать следующие выводы:

*   Как и ожидалось, Simple модели, обученные с нуля всего за 5 эпох, показали значительно более низкое качество по всем метрикам по сравнению с предобученными моделями (Unet base с mAP 0.9684, Segformer base с mAP 0.9711).
*   Simple UNet (Impl base) показал 0.7020 mAP, а Simple ViT (Impl base) - 0.7528 mAP. Эти значения очень низки, что говорит о том, что простым архитектурам требуется либо гораздо больше данных/эпох, либо предобучение, чтобы выучить полезные признаки для задачи сегментации.
*   Интересно отметить, что Simple ViT (Impl base), несмотря на свою упрощенность, на базовом обучении показал немного лучшие результаты (mAP 0.7528) по сравнению с Simple UNet (Impl base) (mAP 0.7020).
*   Основная причина низкой производительности имплементированных моделей на данном этапе - **отсутствие предобучения** на большом датасете, таком как ImageNet, которое дает предобученным моделям сильную базу для переноса знаний.


### Добавление техник из улучшенного бейзлайна к упрощенным моделям

Теперь применим к имплементированным моделям те техники, которые оказались наиболее успешными для предобученных бейзлайнов: **аугментацию данных** и **улучшенные гиперпараметры (пониженный LR, увеличенное число эпох, AdamW для ViT)**. Цель - проверить, насколько эти техники могут улучшить обучение моделей с нуля и сократить разрыв с предобученными моделями.

In [46]:
# Используем параметры и даталоадеры из "Улучшенного бейзлайна"
IMPROVED_SIMPLE_EPOCHS = FINAL_EPOCHS # 10 эпох
IMPROVED_SIMPLE_LR = FINAL_LEARNING_RATE # 0.00005
train_loader_improved_simple = train_loader_improved # Аугментированный даталоадер

print("Настройки обучения имплементированных моделей (улучшенный): Аугментация, LR={IMPROVED_SIMPLE_LR}, Epochs={IMPROVED_SIMPLE_EPOCHS}".format(IMPROVED_SIMPLE_LR=IMPROVED_SIMPLE_LR, IMPROVED_SIMPLE_EPOCHS=IMPROVED_SIMPLE_EPOCHS))

Настройки обучения имплементированных моделей (улучшенный): Аугментация, LR=5e-05, Epochs=10


Обучение и оценка Simple UNet (улучшенный)

In [47]:
# Инициализация Simple UNet (начинаем с чистого листа)
model_unet_simple_improved = SimpleUNet(in_channels=3, out_channels=NUM_CLASSES).to(DEVICE)
optimizer_unet_simple_improved = optim.Adam(model_unet_simple_improved.parameters(), lr=IMPROVED_SIMPLE_LR)

h_loss_unet_simple_improved, f1_unet_simple_improved, mAP_unet_simple_improved = train_and_evaluate_model(
    model=model_unet_simple_improved,
    train_loader=train_loader_improved_simple,
    val_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer_unet_simple_improved,
    device=DEVICE,
    epochs=IMPROVED_SIMPLE_EPOCHS,
    num_classes=NUM_CLASSES,
    ignore_index=IGNORE_INDEX,
    model_name="Simple UNet (Impl improved)"
)


--- Training Simple UNet (Impl improved) ---
EPOCH 1/10
  Train Loss: 0.6314
EPOCH 2/10
  Train Loss: 0.5487
EPOCH 3/10
  Train Loss: 0.5119
EPOCH 4/10
  Train Loss: 0.4921
EPOCH 5/10
  Train Loss: 0.4648
EPOCH 6/10
  Train Loss: 0.4449
EPOCH 7/10
  Train Loss: 0.4247
EPOCH 8/10
  Train Loss: 0.4142
EPOCH 9/10
  Train Loss: 0.4006
EPOCH 10/10
  Train Loss: 0.3956
Training Simple UNet (Impl improved) finished.
Evaluating on test set...
  Hamming Loss (Pixel Error): 0.1686
  F1 Score (Weighted): 0.8269
  mAP (mean IoU): 0.6780
--------------------


Обучение и оценка Simple ViT (улучшенный)

In [48]:
# Инициализация Simple ViT (начинаем с чистого листа)
model_vit_simple_improved = SimpleViTSegmentation(
    image_size=INPUT_DIM[0],
    patch_size=vit_patch_size,
    num_classes=NUM_CLASSES,
    dim=vit_dim,
    depth=vit_depth,
    num_heads=vit_heads,
    mlp_ratio=vit_mlp_ratio,
    channels=3
).to(DEVICE)

# Используем AdamW для трансформера с пониженным LR и weight decay
optimizer_vit_simple_improved = optim.AdamW(model_vit_simple_improved.parameters(), lr=IMPROVED_SIMPLE_LR, weight_decay=0.01)

h_loss_vit_simple_improved, f1_vit_simple_improved, mAP_vit_simple_improved = train_and_evaluate_model(
    model=model_vit_simple_improved,
    train_loader=train_loader_improved_simple,
    val_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer_vit_simple_improved,
    device=DEVICE,
    epochs=IMPROVED_SIMPLE_EPOCHS,
    num_classes=NUM_CLASSES,
    ignore_index=IGNORE_INDEX,
    model_name="Simple ViT (Impl improved)"
)


--- Training Simple ViT (Impl improved) ---
EPOCH 1/10
  Train Loss: 0.5027
EPOCH 2/10
  Train Loss: 0.3998
EPOCH 3/10
  Train Loss: 0.3775
EPOCH 4/10
  Train Loss: 0.3628
EPOCH 5/10
  Train Loss: 0.3487
EPOCH 6/10
  Train Loss: 0.3409
EPOCH 7/10
  Train Loss: 0.3325
EPOCH 8/10
  Train Loss: 0.3268
EPOCH 9/10
  Train Loss: 0.3213
EPOCH 10/10
  Train Loss: 0.3149
Training Simple ViT (Impl improved) finished.
Evaluating on test set...
  Hamming Loss (Pixel Error): 0.1339
  F1 Score (Weighted): 0.8639
  mAP (mean IoU): 0.7378
--------------------


### Сравнение результатов улучшенных моделей

In [49]:
results_improved = {
    'Model': ['Unet (improved)', 'Simple UNet (Impl improved)', 'Segformer (improved)', 'Simple ViT (Impl improved)'],
    'Hamming Loss (Pixel Error)': [h_loss_unet_improved, h_loss_unet_simple_improved, h_loss_segformer_improved, h_loss_vit_simple_improved],
    'F1 Score (Weighted)': [f1_unet_improved, f1_unet_simple_improved, f1_segformer_improved, f1_vit_simple_improved],
    'mAP (mean IoU)': [mAP_unet_improved, mAP_unet_simple_improved, mAP_segformer_improved, mAP_vit_simple_improved]
}

df_improved_comparison = pd.DataFrame(results_improved).round(4)
print("\nСравнение улучшенных моделей (предобученные vs имплементированные):")
print(tabulate(df_improved_comparison, headers='keys', tablefmt='pipe'))


Сравнение улучшенных моделей (предобученные vs имплементированные):
|    | Model                       |   Hamming Loss (Pixel Error) |   F1 Score (Weighted) |   mAP (mean IoU) |
|---:|:----------------------------|-----------------------------:|----------------------:|-----------------:|
|  0 | Unet (improved)             |                       0.0113 |                0.9887 |           0.9752 |
|  1 | Simple UNet (Impl improved) |                       0.1686 |                0.8269 |           0.6780 |
|  2 | Segformer (improved)        |                       0.0119 |                0.9881 |           0.9738 |
|  3 | Simple ViT (Impl improved)  |                       0.1339 |                0.8639 |           0.7378 |


### Выводы

На основе сравнения метрик всех четырех финальных вариантов моделей можно сделать следующие выводы:

*   После применения техник из улучшенного бейзлайна (аугментация, пониженный LR, увеличенное число эпох, AdamW для трансформеров), качество предобученных моделей значительно улучшилось, достигнув mAP 0.9752 для Unet (improved) и 0.9738 для Segformer (improved).
*   Однако, имплементированные Simple модели (Simple UNet Impl improved: 0.6780 mAP, Simple ViT Impl improved: 0.7378 mAP), при обучении с нуля с теми же "улучшенными" гиперпараметрами, показали **снижение** качества по сравнению с их базовым обучением (Simple UNet Impl base: 0.7020 mAP, Simple ViT Impl base: 0.7528 mAP). Этот неожиданный результат говорит о том, что гиперпараметры и техники, оптимальные для дообучения предобученных моделей (где LR должен быть низким для тонкой настройки), не подходят для обучения простых моделей с нуля. Модели, обучаемые с нуля, вероятно, требуют более высокого Learning Rate и, возможно, других аугментаций или регуляризации на начальных этапах обучения.
*   Даже с "улучшенными" техниками обучения, имплементированные Simple модели по-прежнему существенно уступают предобученным моделям из библиотеки. Это убедительно демонстрирует **критическую важность предобучения** для задач сегментации на сравнительно небольших датасетах, таких как Oxford-IIIT Pet. Предобученные модели обладают сильными базовыми признаками, которые легко адаптируются под новую задачу, в то время как модели, обучаемые с нуля, требуют огромного объема данных и/или значительно большего времени/ресурсов для достижения схожего уровня понимания изображений.
*   Среди предобученных моделей, Unet (improved) показал немного лучший результат (0.9752 mAP) по сравнению с Segformer (improved) (0.9738 mAP) на данном датасете с использованными параметрами. Это может говорить о том, что классическая сверточная архитектура с эффективным энкодером (ResNet34) хорошо справляется с этой задачей, возможно, лучше адаптируясь к локальным особенностям изображения, чем трансформерная модель с более глобальным вниманием (Segformer mit_b0).
*   Среди имплементированных моделей, Simple ViT (Impl improved) показал немного лучшие метрики (0.7378 mAP) по сравнению с Simple UNet (Impl improved) (0.6780 mAP), хотя обе модели показали снижение качества по сравнению с базовым обучением. Это может быть связано с тем, что даже упрощенная трансформерная архитектура имеет потенциал для выучивания более мощных представлений, но ее обучение с нуля более чувствительно к выбору гиперпараметров и техник, чем упрощенная сверточная архитектура.
*   В целом, лабораторная работа показала, что для достижения высокого качества семантической сегментации на Oxford-IIIT Pet наиболее эффективным подходом является дообучение **предобученных моделей** из библиотеки, сочетая **аугментацию данных** и **тонкую настройку гиперпараметров (особенно Learning Rate и количество эпох)**. Имплементированные модели, хотя и могут улучшаться от правильных техник обучения с нуля, не могут преодолеть разрыв в производительности без предобучения или значительно более сложной архитектуры и большего количества данных/ресурсов для обучения с нуля. Применение техник fine-tuning к обучению с нуля может даже навредить, как показали результаты с Simple моделями.
